<a href="https://colab.research.google.com/github/DaeSeokSong/LSTM-PPoA/blob/main/Predict_Price_of_Agricultural.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 당년 농산물 가격 예측 LSTM 모델

### Google Drive Mount

In [10]:
from google.colab import drive
drive.mount('/content/gdrive')

%cd /content/gdrive/MyDrive/DeepLearning/Project/PPoA
!ls -al

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/MyDrive/DeepLearning/Project/PPoA
total 14
drwx------ 2 root root 4096 Nov 24 06:07  Dataset
-rw------- 1 root root 9526 Nov 24 06:28 'Predict Price of Agricultural.ipynb'


### Import


In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense
from keras.utils import *
from sklearn.preprocessing import *

# Function

In [ ]:
def Normalization() :
    

### Data read

In [42]:
f_name = '2008-2020_cabagePrice.csv'
df = pd.read_csv('./Dataset/'+f_name, index_col=0)

"""
[학습 변수(데이터)]
    - 기상 변수 (기온, 강수량, 습도, 풍량 적설량), 변수에 대해 최저값, 최고값, 평균값, 중간값 사용
    - 기타 변수 (경유, 물가 상승률, 전년 수확량, 전년 수입량, 전년 재배면적)
"""

''' 기상변수 '''
# 강수량
precipitation_avg = df['전국 강수량 평균']
precipitation_chg = df['강수량 증감률']

''' 기타변수 '''
# 재배면적
cultivation_area = df['면적']
cultivation_area_chg = df['면적 증감율']
# 수입량
amount_importation = df['총 수입량']
amount_importation_chg = df['총 수입량 증감률']
# 생산량(수확량)
production = df['생산량']
production_chg = df['생산량 증감률']
# 물가 상승률
price_avg = df['전국 가격 평균']
price_chg = df['가격 증감률']

df.head()

,전국 강수량 평균,강수량 증감률,합계 일조량,평균기온,평균최저기온,최고기온,총 수입량,총 수입량 증감률,Yuan 환율,Yuan 환율 증감률,달러 환율,달러 환율 증감률,면적,면적 증감율,생산단수,생산량,생산량 증감률,전국 가격 평균,가격 증감률,최대,최대가격 증감률,최소,최소가격 증감률,표준편차,변동계수,진폭계수,공급량,공급량 증감률
2020,1627.8,30.59%,"2,394.50",13.7,9.2,35.4,"281,187",-8.12%,6.90,0%,"1,180",1%,"13,854",26.30%,"9,670","1,339,742",26%,"3,268",-26.53%,"4,664",-5.80%,"2,722",-3%,554,16.95,0.71,"1,620,929",18.66%
2019,1246.5,-14.09%,"2,474.30",14.0,9.2,36.0,"306,049",5.26%,6.91,4%,"1,166.11",6%,"10,968",-17.60%,"9,664","1,059,925",-25%,"4,448",50.52%,"4,951",47%,"2,857",3%,454,10.21,0.73,"1,365,974",-19.41%
2018,1451.0,50.72%,"2,542.10",13.5,8.8,39.4,"290,742",5.48%,6.62,-2%,"1,100.58",-3%,"13,313",-2.60%,"10,547","1,404,151",3%,"2,955",9.77%,"3,369",10.40%,"2,773",16.30%,150,5.08,0.21,"1,694,893",3.46%
2017,962.7,-30.16%,"2,632.00",13.4,8.5,35.9,"275,631",8.76%,6.76,2%,"1,130.48",-3%,"13,674",19.60%,"9,965","1,362,621",21%,"2,692",-25.72%,"3,051",-27.80%,"2,384",12.90%,146,5.42,0.28,"1,638,252",18.58%
2016,1378.5,33.98%,"2,440.10",14.0,9.4,37.6,"253,432",13.08%,6.64,7%,"1,160.41",3%,"11,429",-10.20%,"9,871","1,128,122",-21%,"3,624",69.50%,"4,224",85.10%,"2,111",12.60%,595,16.42,1.00,"1,381,554",-16.78%


### Data preprocessing

In [ ]:
chgs = [precipitation_chg, cultivation_area_chg, amount_importation_chg, price_chg]

for chg_list in chgs :
    chg_idxs = chg_list.index
    for idx, e in enumerate(chg_list) :
        chg_list[chg_idxs[idx]] = float(e[0:len(e)-1])

### Modeling

In [ ]:
model = Sequential()

model.add(LSTM(
    1, # 해당 층의 노드 개수
    input_shape=(50,1), # input_shape=?
    return_sequences=True)) # return_sequences == 각 시퀀스를 출력할지
model.add(Dropout(0.01)) # 과적합 방지용 Dropout 20%(==0.2)

model.add(LSTM(
    100, # 노드 100개
    return_sequences=False))
model.add(Dropout(0.01))

model.add(Dense(1, activation='sigmoid')) # 활성화(Activation)함수 = sigmoid
model.compile(loss='mse', optimizer='rmsprop')

### Learning

### Prediction